# Object Detection(객체 탐지)
<hr>

- 두 가지 작업(task)을 포함한 것을 `Object Detection`이라고 함
    - Localization(위치결정): 물체(객체)의 `위치 정보`를 찾는 것
        - `위치 정보`: 더 정확하게는 물체(객체)가 있을 것 같은 `경계상자`를 찾는 것
    - Classification(분류): 해당 상자에 포함되어 있는 물체(객체)가 `무엇인지 분류`하는 것
    - Localization과 Classification을 수행하기 위해서 일반적으로 두 단계가 필요
        - 1단계. 물체(객체)가 있을 만한 `경계상자`를 추출
            - `경계상자`: resions of interest, `ROI`(관심 영역)
        - 2단계. 추출된 `ROI`를 이용하여 Localization과 Classification을 수행
    - Object Detection 알고리즘은 크게 두 가지로 구분됨
        - `One Stage Detectors(or Algorithm)`
            - 위 두 단계를 한 번에 수행
            - 알고리즘: SSD(Single Shot Detection), YOLO(You Only Look Once) 등
            - 특징: `two stage detectors`에 비해 상대적으로 연산 속도가 빠르고, 정확도는 낮지만 큰 차이는<br>
            없거나 더 좋은 경우도 있음
            - 참고사항: 대표적인 두 알고리즘 중 SSD가 YOLO를 보완하여 만들었고, SSD가 작은 사이즈의 객체를<br>
            탐지하는 성능이 YOLO보다 우수하다고 알려져 있음
        - `Two Stage Detectors(or Algorithm)`
            - 위 두 단계를 구분하여 수행
            - 알고리즘: `R-CNN` family(R-CNN, Fast R-CNN, Faster R-CNN, `Mask R-CNN`)
                - `R-CNN`: Region-Based Convolutional Neural Network(영역 기반 합성곱 신경망)
                - `Mask R-CNN`: 물체(객체) 탐지도 수행하지만 주 목적은 `Image Segmentation`(이미지 분할)
            - 특징: `one stage detectors`에 비해 상대적으로 연산 속도가 느리고, 정확도는 높다.
- Object Detection은 지도학습과 비지도학습 중 `지도학습`에 해당함
    - Localization과 Classification 이라는 두가지 작업에 해당하기 때문에 각각에 대한 레이블이 필요함

### SSD(Single Shot Detection)
<hr>

#### Overview
- 지도학습의 한 종류이기 때문에 이미지 안에 정답 경계상자, 클래스 정보가 담긴 `정답 이미지`가 존재해야 하며,<br>
알고리즘이 `정답 경계상자`를 찾고, 각 `정답 경계상자`에 속한 객체를 예측해야 함
    - `정답 경계상자`: Ground Truth Bounding box, `GTBB`
- 이미지마다 존재하는 객체의 수, 객체의 위치, 객체의 크기, 객체의 형태가 다르다. 그러므로<br>
아무것도 없는 상태의 이미지에서 위치, 크기, 형태가 각각 다른 `GTBB`를 예측하는 것은 어렵기 때문에<br>
`기본상자`라는 것을 사용
    - `기본상자`: Anchor box or Default box
    - `참고사항`: Faster R-CNN에서 이 `anchor boxes`를 활용한 `RPN(Resion Proposal Network)`방식을 처음 도입,<br>
    이후 SSD에서 `RPN` 단계를 Classification과 `통합하여 수행함`
- Default box(기본상자)
    - 다양한 위치, 크기, 형태의 default boxes를 사용하여 default boxes를 기준점으로 GTBB가 default boxes로부터<br>
    얼마나 떨어져 있는지(`offset` 정도), 그리고 해당 GTBB에 존재하는 객체가 무엇인지를 예측함(두 작업 동시 수행)
        - bounding box regression: `offset` 정도 예측, Classification: 물체(객체)의 `클래스 분류`
    - SSD는 default boxes를 원본 이미지에서 설정하지 않고 `사전학습모형(VGG-16)`과 `추가적인 합성곱 레이어`를 이용하여<br>
    특징 맵(feature map)을 추출하고, 추출된 특징 맵(feature map)에서 default boxes를 설정하여 사용함
        - `사전 학습 모형`: backbone network라고 일컬음
- Feature map(특징 맵)
    - SSD는 Object Detection을 위해 사용하는 `특징 맵의 수가 6개`
        - 1차적으로 VGG-16과 같은 `사전학습모형(backbone network)`을 사용하여 `6개`의 특징 맵을 추출하고,<br>
        그 중 마지막 `6번째 특징 맵`을 사용(Conv4_3)
        - `추가적인 합성곱 레이어`를 사용하여 다시 `6개`의 특징 맵을 추출하고, 그 중 `5개`를 사용
        - 결론적으로 `사전학습모형(1개) + 합성곱 레이어(5개) = 6개의 크기가 각각 다른 특징 맵`을 사용하고,<br>
        이를 이용하여 물체(객체)를 탐지하여 다양한 크기의 물체(객체)를 잘 찾을 수 있음(`multi-scale object detection`)
            - `multi-scale object detection`은 SSD의 가장 큰 장점 중 하나
    - <img src="markdown/SSD_architecture.png" width="800">
<hr>

- Feature map에 Default boxes 설정
    - 하나의 특징 맵은 격자판(grid)하고 생각할 수 있음
        - 격자판(grid)에는 격자판(grid) 크기 만큼의 `cell`이 존재함
            - e.g., 25 cells exists in 5x5 grid(5x5=25)
    - 각 feature map은 output으로 연결되는 또하나의 convolution layer를 거쳐 각각의 `cell`에 대하여<br>
    `4개 or 6개`의 default boxes를 설정하고, 그에 `대응`하는 `cell`의 순서 & `해당` default box에서 `GTBB`의<br>
    `offset(cx, cy, w, h)`과 `GTBB`안에 존재하는 물체의 `confidences`값을 예측함
        - SSD는 총 21개(`background 1 + object class 20`)의 클래스를 사용하기 때문에 `confidences=21`이 됨
        - 이 때 사용하는 convolution layer는 filter=3 x 3 x (k x (classes+4)), padding=1, stride=1
            - `k`: 4 or 6
        - 위 SSD 구조 이미지에서 `Conv4_3[1], Conv10_2[5], Conv11_2[6]`에서는 `cell 당 4개`의 default boxes를,<br>
        `Conv7[2], Conv8_2[3], Conv9_2[4]`에서는 `cell 당 6개`의 default boxes를 설정함
            - result of Conv4_3: 38 x 38 x (`4` x (classes + 4)), `5776` x (classes + 4)
            - result of Conv7: 19 x 19 x (`6` x (classes + 4)), `2166` x (classes + 4)
            - result of Conv8_2: 10 x 10 x (`6` x (classes + 4)), `600` x (classes + 4)
            - result of Conv9_2: 5 x 5 x (`6` x (classes + 4)), `125` x (classes + 4)
            - result of Conv10_2: 3 x 3 x (`4` x (classes + 4)), `36` x (classes + 4)
            - result of Conv11_2: 1 x 1 x (`4` x (classes + 4)), `4` x (classes + 4)
            - 클래스 당 `8732`개의 default boxes 설정 & 그에 대응하는 각각의 default box 자리에서<br>
            정답을 예측(detect)함
                - 이후 모델은 `비최대 억제(Non-maximum Suppression)` 과정을 거쳐 `confidence`값이 가장 높은<br>
                `bounding box`를 `test output(결과)`을 내보냄

#### Training
<hr>

- Choosing scales and aspect ratio for default boxes(기본 상자를 설정하기 위한 규모와 종횡비 선택)
    - 각 셀에 대한 default boxes의 설정 방법
        - 보다 다양한 형태의 물체를 찾기 위해 다양한 형태의 default boxes를 사용하며, default boxes의 형태는<br>
        `scaling factor`와 `aspect ratio`에 의해 결점됨
        - Scaling factor
            - default boxes의 `상대적인 크기`를 결정
            - k번째 feature map에 대한 scaling factor는 아래와 같이 계산됨
                - $S_k = S_{min} + \frac{S_{max}-S_{min}}{m-1}(k-1),\quad k\in [1, m]$
                - $S_{min} = 0.2,\ S_{max} = 0.9$
                - $m$은 feature map의 개수를 의미하며 SSD는 총 `6개의 feature map`을<br>
                사용하기 때문에 $m$의 최댓값은 `6`
                    - $k=1$, $S_k$ = 0.2 $(S_{min})$
                    - $k=2$, $S_k$ = 0.34
                    - $k=3$, $S_k$ = 0.48
                    - $k=4$, $S_k$ = 0.62
                    - $k=5$, $S_k$ = 0.76
                    - $k=6$, $S_k$ = 0.9 $(S_{max})$
        - Aspect ratio
            - default boxes의 `가로세로 비율(종횡비)`을 결정
            - cell당 default boxes가 `4개`인 경우
                - $a_r\in \{1,2,\frac{1}{2}\}$ -> `3개` + `extra 1개`
            - cell당 default boxes가 `6개`인 경우
                - $a_r\in\{1,2,3,\frac{1}{2},\frac{1}{3}\}$ -> `5개` + `extra 1개`
            - `extra scaling factor` $S'_k$를 사용하여 $a_r=1$ 인 default box보다 조금 더 큰 사이즈의<br>
            default box를 추가로 사용함
                - $S'_k = \sqrt {S_k S_{k+1}}$
        - `k번째` Feature map(grid)에 대해 `1개 cell`에 존재하는 `default boxes`의 크기(w, h)는<br>
        아래와 같이 결정됨
            - $(w^a_k,\ h^a_k)=(S_k\sqrt{a_r},\ S_k\frac{1}{\sqrt{a_r}})$
            - $(w^a_k,\ h^a_k)=(S'_k,\ S'_k),\quad\mathsf{where}\ \ S'_k=\sqrt{S_k S_{k+1}}$
            - $w^a_k,\ h^a_k$ = `k번째` feature map(grid)의 `a(종횡비)`가 적용된 default boxes의 `너비와 높이`
            - $S_k$ = `k번째` feature map의 `scaling factor`
            - $a_r$ = default boxes의 `aspect ratio(종횡비)`
            - $S'_k$ = 추가적인 `scaling factor`
        - Default box 가로세로 크기의 모든 경우의 수
            - $k=1$일 때
                - (0.2, 0.2)
                - (0.28, 0.14)
                - (0.35, 0.12)
                - (0.14, 0.28)
                - (0.12, 0.35)
                - (0.26, 0.26)
            - $k=2$일 때
                - (0.34, 0.34)
                - (0.48, 0.24)
                - (0.59, 0.2)
                - (0.24, 0.48)
                - (0.2, 0.59)
                - (0.4, 0.4)
            - $k=3$일 때
                - (0.48, 0.48)
                - (0.68, 0.34)
                - (0.83, 0.28)
                - (0.34, 0.68)
                - (0.28, 0.83)
                - (0.55, 0.55)
            - $k=4$일 때
                - (0.62, 0.62)
                - (0.88, 0.44)
                - (1.07, 0.36)
                - (0.44, 0.88)
                - (0.36, 1.07)
                - (0.69, 0.69)
            - $k=5$일 때
                - (0.76, 0.76)
                - (1.07, 0.54)
                - (1.32, 0.44)
                - (0.54, 1.07)
                - (0.44, 1.32)
                - (0.83, 0.83)
            - $k=6$일 때
                - (0.9, 0.9)
                - (1.27, 0.64)
                - (1.56, 0.52)
                - (0.64, 1.27)
                - (0.52, 1.56)
                - `(0.97, 0.97)`-> `k가 7`이어야 계산이 가능하기 때문에 존재하지 않음(?)
        - 각 cell마다 default boxes가 위치할 `중심점(cx, cy)` 설정
            - 각각의 `cell`마다 `4개` or `6개`의 default boxes가 위치하려면 `해당 cell의 중심점`이 필요하며,<br>
            그 `중심점`은 아래와 같이 계산됨
            - $(\frac{i+0.5}{|f_{k}|},\ \frac{j+0.5}{|f_{k}|}),\quad{\scriptstyle\text{where }}|f_{k}|={\scriptstyle\text{k번째 square feature map의 한 변의 길이}},\quad i,j\in[0,|f_{k}|)$
        - 정리하자면 특징 맵의 순서 k가 [1,6]일 때, 각 특징맵에 존재하는 m x n 만큼의 cells 에서 1개의 cell에 설정되는<br>
        default boxes는 특징 맵의 순서에 따라 4개 or 6개가 설정되고, 각 default box의 (cx, cy, w, h)는<br>
        $(\frac{i+0.5}{|f_{k}|},\ \frac{j+0.5}{|f_{k}|},\ w^a_k,\ h^a_k)$으로 정해짐
<hr>

- Matching strategy(매칭 전략)
    - 훈련이 진행되는 동안 어떠한 `default box`가 `정답을 탐지`한 것에 해당하는지 결정하고 그에 따라서 모델을 훈련 시켜야 함
        - 각 `default box`는 `(cx, cy, w, h)`값을 가지고 있고, 해당 값을 이용하여 `ground truth box`와 겹치는 정도를<br>
        계산하기 위해 `Jaccard overlap` 지표를 사용함
            - `Jaccard overlap`: object detector의 성능 평가 지표로 사용되고,<br>
            `Intersection over Union(IoU)`라고도 칭하며, `0~1` 사이의 실수값을 가짐
            - <img src="markdown/intersection_over_union.png" width="400">
        - 먼저 각 `ground truth box`에 대하여 `IoU값`이 가장 큰 `default box`를 찾고, 해당 `ground truth box`와 매칭함(`ground truth box`는 1개 이상)
        - 이후 `ground truth box`를 할당 받지 못 한 `default boxes` 중 `IoU값`이 `0.5`보다 큰 `default boxes` 또한<br>
        해당 `ground truth box`와 매칭함
            - 만약 임의의 `default box`에서 `IoU값`이 `0.5`보다 큰 `ground truth box`가 2개 이상 존재한다면 그 중 가장 큰<br>
            `IoU값`을 갖는 `ground truth box`와 매칭함
        - 이러한 매칭 전략은 각 특징 맵에서 `maximum overlap`이 발생하는 것만 선택하도록 요구하는 방향으로 학습되는<br>
        대신에 여러 `overlapping default boxes`에 대해 네트워크가 `좋은 성능을 발휘`할 수 있게 함
        - 매칭 작업이 모두 끝난 `default boxes`는 `positives` = 매칭된 `default boxes`,<br>
        `negatives` = 매칭되지 않은 `default boxes`로 분류됨
<hr>

- Training objective(훈련 목표)
    - 네트워크는 두 가지의 값을 학습하기 위해 `손실함수(loss function)`를 계산함
        - 먼저 $x_{ij}^p=\{1,0\}$ 를 `i`번째 default box와 클래스가 `p`인 `j`번째 ground truth box와 매칭되는<br>
        지표로써 정의하고, 위의 `매칭 전략`에 따라 $\sum_i x_{ij}^p \ge 1$의 값을 가질 수 있음(`positives`가 적어도 1개 이상)
        - `localization loss`는 `매칭`된 `i`번째 `default box(d)`와 같은 순번인 `i`번째 `predicted box(l)`를 고려함<br>
        <br>
            - $\quad\quad L_{loc}(x,l,g)=\sum_{i\in Pos}^N\sum_{m\in\{cx,cy,w,h\}} x_{ij}^k\mathsf{smooth_{L1}}(l_i^m - \hat g_j^m)$<br>
            <br>
            - $l=\mathsf{{\color{blue}predicted\ box}}$
            - $g=\mathsf{ground\ truth\ box}$
            - $\hat g_j^{cx}=(g_j^{cx}-d_i^{cx})/d_i^w\quad\hat g_j^{cy}=(g_j^{cy}-d_i^{cy})/d_i^h$
            - $\hat g_j^{w}=log(\frac{g_j^w}{d_i^w})\quad\hat g_j^{h}=log(\frac{g_j^h}{d_i^h})$
            - 위 식을 해석하자면 `positives` 중 $x_{ij}^k=1$인 `default box`와 같은 위치의 `predicted box`와<br>
            클래스가 `k`인 `ground truth box`의 `offset loss(불일치 오차)`을 계산함
                - 초기 예측값은 default box 위치와 같은 값을 예측하는 것으로 시작할 것이라고 예상됨<br>
                <br>
        - `confidence loss`는 `positives`와 `negatives` 둘다 사용하며 `positives`에 대해서는 다중 클래스에 대한<br>
        `softmax loss`를, `negatives`에 대해서는 `background` 클래스에 대한 `confidence`값을 고려함<br>
        <br>
            - $\quad\quad L_{conf}(x,c)=-\sum_{i\in Pos}^N x_{ij}^p log(\hat c_i^p)-\sum_{i\in Neg}log(\hat c_i^0)\quad\mathsf{where}\quad\hat c_i^p=\frac{exp(c_i^p)}{\sum_p exp(c_i^p)}$<br>
            <br>
            - $\hat c_i^p=$ 다중 클래스에 대한 p 클래스의 `softmax loss`
            - $\hat c_i^0=\frac{exp(c_i^0)}{\sum_p exp(c_i^0)}$ 다중 클래스에 대한 0 클래스의 `softmax loss`
            - 위 식 또한 해석해보자면 `positives` 중 $x_{ij}^p=1$인 `default box`와 같은 위치의 `predicted box`와<br>
            클래스가 `p`인 `ground truth box`의 `confidence loss(신뢰도 오차)`를 계산하고, `negatives` 또한 같은 방법으로 계산하여 마지막으로 더해줌<br>
            <br>
        - 전체적인 `objective loss function`은 위 두가지 `loss`값의 `가중합(weighted sum)`으로 구성됨<br>
        <br>
            - $\quad\quad L(x,c,l,g)=\frac{1}{N}(L_{conf}(x,c)+\alpha L_{loc}(x,l,g))$<br>
            <br>
            - $N=$ 매칭 지표$(x_{ij}^p)$에 의해 매칭된 default box의 수
            - $\alpha=$ 교차검증에 의해 얻어진 `1`로 설정함
<hr>

- Hard negative mining
    - 매칭 단계 이후, 대부분의 `default box`는 `negatives`이고, 특히 가용 `default box`가 많을 경우엔<br>
    이 비율이 더욱 커지며, 이는 positive와 negative 훈련 샘플 간에 심각한 `불균형(imbalance)`을 초래함
    - 모든 `negatives`를 쓰는 것 대신에 각 default box에 대해 $L_{conf}$(`confidence loss`)값이 `높은` 것을 사용하여<br>
    정렬하고, 그 중 가장 위부터 선별(선택)하여 $L_{conf}$(`confidence loss`)를 계산할 때 `negatives`와 `positives`의<br>
    비율이 `최대 3:1`이 되도록 함
        - 논문에서 저자들은 이러한 방법이 더 빠른 최적화와 더 안정적인 훈련으로 이어진다는 것을 발견했다고 서술함
        - Hard negative mining 기법은 R-CNN, OHEM 등 다른 알고리즘에서도 사용되었음
        - Hard negative mining은 모델이 예측에 실패하는 어려운(hard) sample들을 모으는 기법으로,<br>
        Hard negative mining을 통해 수집된 데이터를 활용하여 모델을 보다 강건하게 학습시키는 것이 가능해짐
            - 예를 들어 이미지에서 사람의 안면의 위치를 탐지하는 모델을 학습시킨다고 할 때, 사람의 안면은<br>
            `positive sample`이며, 그외의 배경은 `negative sample`. 이 때 모델이 배경이라고 예측했으며,<br>
            실제로 배경인 bounding box는 `True Negative sample`에 해당함. 반면에 모델이 안면이라고 예측했지만,<br>
            실제로 배경인 경우는 `False Positive sample`에 해당함. 모델은 주로 `False Positive`라고 예측하는<br>
            오류를 주로 범함. 이는 객체 탐지 시, 객체의 위치에 해당하는 `positive sample`보다 배경에 해당하는<br>
            `negative sample`이 훨씬 많은 `클래스 불균형(class imbalance)`으로 인해 발생하게 됨.<br>
            이러한 문제를 해결하기 위해 모델이 잘못 판단한 `False Positive sample`을 학습 과정에서 추가하여<br>
            재학습하면 모델은 보다 강건해지며, `False Positive`라고 판단하는 오류가 줄어들게 됨

In [22]:
import tensorflow as tf
import numpy as np

x = np.array([
    [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]
    ])
print(x.shape)
print(x)
print(tf.cast(tf.reduce_max(x, -1), tf.float32))

(1, 6, 6)
[[[1 0 0 0 0 0]
  [0 1 0 0 0 0]
  [0 0 1 0 0 0]
  [0 0 0 1 0 0]
  [0 0 0 0 1 0]
  [0 0 0 0 0 1]]]
tf.Tensor([[1. 1. 1. 1. 1. 1.]], shape=(1, 6), dtype=float32)


In [30]:
from math import *
k = 5
S = [0.2,0.34,0.48,0.62,0.76,0.9,1.04]
a = [1,2,3,1/2,1/3]
for i in range(6):
    if i == 5:
        print((round(sqrt(S[k]*S[k+1]),2), round(sqrt(S[k]*S[k+1]),2)))
        break
    print((round(S[k]*sqrt(a[i]),2), round(S[k]*(1/sqrt(a[i])),2)))

(0.9, 0.9)
(1.27, 0.64)
(1.56, 0.52)
(0.64, 1.27)
(0.52, 1.56)
(0.97, 0.97)


In [31]:
import math


S_MIN = 0.2
S_MAX = 0.9
def get_scales(m):
    scales = []    
    for k in range(1, m+1):
        scales.append(round((S_MIN + (S_MAX - S_MIN) / (m - 1) * (k - 1)), 2)) 
    return scales
get_scales(6)
((()))
RATIOS = [1, 2, 3, 0.5, 0.33]
def get_width_height(scales):
    width_heights = []
    for k, scale in enumerate(scales):
        print(f'k: {k+1} scale: {scale}')
        width_height_per_scale = []
        for ratio in RATIOS:
            width = min(round((scale *  math.sqrt(ratio)), 2), 1)
            height = min(round((scale /  math.sqrt(ratio)), 2), 1)
            width_height_per_scale.append((width, height))
            print(f'widht: {width} height: {height}')
        if k < len(scales) - 1:
            extra_sacle = round(math.sqrt(scale * scales[k+1]), 2)
            width_height_per_scale.append((extra_sacle, extra_sacle))
            print(f'width: {extra_sacle} height: {extra_sacle}')
        width_heights.append(width_height_per_scale)
        print('')
    return width_heights

scales = get_scales(6)
width_heights = get_width_height(scales)

k: 1 scale: 0.2
widht: 0.2 height: 0.2
widht: 0.28 height: 0.14
widht: 0.35 height: 0.12
widht: 0.14 height: 0.28
widht: 0.11 height: 0.35
width: 0.26 height: 0.26

k: 2 scale: 0.34
widht: 0.34 height: 0.34
widht: 0.48 height: 0.24
widht: 0.59 height: 0.2
widht: 0.24 height: 0.48
widht: 0.2 height: 0.59
width: 0.4 height: 0.4

k: 3 scale: 0.48
widht: 0.48 height: 0.48
widht: 0.68 height: 0.34
widht: 0.83 height: 0.28
widht: 0.34 height: 0.68
widht: 0.28 height: 0.84
width: 0.55 height: 0.55

k: 4 scale: 0.62
widht: 0.62 height: 0.62
widht: 0.88 height: 0.44
widht: 1 height: 0.36
widht: 0.44 height: 0.88
widht: 0.36 height: 1
width: 0.69 height: 0.69

k: 5 scale: 0.76
widht: 0.76 height: 0.76
widht: 1 height: 0.54
widht: 1 height: 0.44
widht: 0.54 height: 1
widht: 0.44 height: 1
width: 0.83 height: 0.83

k: 6 scale: 0.9
widht: 0.9 height: 0.9
widht: 1 height: 0.64
widht: 1 height: 0.52
widht: 0.64 height: 1
widht: 0.52 height: 1

